# Performance Analysis
The experimental results are obtained by running the following commands:
```bash
bin/radice-perf -r 32 --trace baseline --topology base --engine opendc \
                -P workload=baseline -P topology=baseline
bin/radice-perf -r 32 --trace baseline --topology base --engine cloudsim-plus \
                -P workload=baseline -P topology=baseline
                
bin/radice-perf -r 32 --trace baseline-50% --topology base --topology-scale 0.5 --engine opendc \
                -P workload=baseline-50% -P topology=baseline-50%
bin/radice-perf -r 32 --trace baseline-50% --topology base --topology-scale 0.5 --engine cloudsim-plus \
                -P workload=baseline-50% -P topology=baseline-50%

bin/radice-perf -r 32 --trace baseline-25% --topology base --topology-scale 0.25 --engine opendc \
                -P workload=baseline-25% -P topology=baseline-25%
bin/radice-perf -r 32 --trace baseline-25% --topology base --topology-scale 0.25 --engine cloudsim-plus \
                -P workload=baseline-25% -P topology=baseline-25%

bin/radice-perf -r 32 --trace baseline-10% --topology base --topology-scale 0.1 --engine opendc \
                -P workload=baseline-10% -P topology=baseline-10%
bin/radice-perf -r 32 --trace baseline-10% --topology base --topology-scale 0.1 --engine cloudsim-plus \
                -P workload=baseline-10% -P topology=baseline-10%
```
Note that the CloudSim Plus engine takes about 20 minutes to complete a single baseline workload on recent hardware.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.lines import Line2D
import seaborn as sns
import radice

FIGURE_PATH = 'figures/05-performance'

os.makedirs(FIGURE_PATH, exist_ok=True)
radice.set()

In [ ]:
df = pd.read_parquet('data/perf')
df['duration'] = pd.to_timedelta(df['duration'], unit='ms')
df['mem_usage_mb'] = df['mem_usage'] / 1000000
df

In [ ]:
df.groupby(["workload", "engine"])['duration'].describe()

In [ ]:
df.groupby(["workload", "engine"])['mem_usage_mb'].describe()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=radice.figsize(2), sharey=True)

order=['baseline-5%', 'baseline-10%', 'baseline-25%', 'baseline-50%', 'baseline']
hue_order=['opendc', 'cloudsim-plus']

sns.barplot(data=df, x=df['duration'].dt.seconds, y='workload', hue='engine', orient='h', ax=ax[0], order=order, hue_order=hue_order, edgecolor='black')

ax[0].set_xlabel("Runtime [\si{\second}]")
ax[0].set_ylabel("Workload")

sns.barplot(data=df, x='mem_usage_mb', y='workload',  hue='engine', orient='h', ax=ax[1], order=order, hue_order=hue_order, edgecolor='black')

ax[1].set_xlabel("Memory Usage [\si{\mega\\byte}]")
ax[1].set_ylabel("")
ax[1].xaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

legend = ax[1].get_legend()
legend.set_title("Simulator")
legend.get_texts()[0].set_text('Radice + OpenDC (v2.1)')
legend.get_texts()[1].set_text('CloudSim Plus (v7.1.1)')

ax[0].legend([],[], frameon=False)
ax[0].set_yticklabels(["Baseline (5\\%)", "Baseline (10\\%)", "Baseline (25\\%)", "Baseline (50\\%)", "Baseline (100\\%)"])
ax[0].xaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

fig.align_ylabels(ax)
fig.savefig(os.path.join(FIGURE_PATH, 'performance-comparison.pgf'))

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(2))

data = df[df['engine'] == 'cloudsim-plus']
a = df[df['engine'] == 'cloudsim-plus'].set_index('workload')['duration']
b = df[df['engine'] == 'opendc'].groupby(['workload'])['duration'].mean()

order=['baseline-5%', 'baseline-10%', 'baseline-25%', 'baseline-50%', 'baseline']

sns.barplot(data=data, x=a / b, y=(a / b).index, orient='h', ax=ax, order=order, edgecolor='black', color=sns.color_palette()[0])

ax.set_xlabel("Speedup compared to CloudSim Plus (v7.1.1)")
ax.xaxis.set_major_formatter('{x:0.0f}x')
ax.set_ylabel("Workload")
ax.set_yticklabels(["Baseline (5\%)", "Baseline (10\%)", "Baseline (25\%)", "Baseline (50\%)", "Baseline (100\%)"], ha='left')
ax.get_yaxis().set_tick_params(pad=60)

fig.savefig(os.path.join(FIGURE_PATH, 'runtime-comparison.pgf'))

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(1.8))

order=['baseline-5%', 'baseline-10%', 'baseline-25%', 'baseline-50%', 'baseline']
hue_order=['opendc', 'cloudsim-plus']

sns.barplot(data=df, x='mem_usage_mb', y='workload',  hue='engine', orient='h', ax=ax, order=order, hue_order=hue_order, edgecolor='black')

ax.set_xlabel("Memory Usage [\si{\mega\\byte}]")
ax.set_ylabel("Workload")
ax.set_yticklabels(["Baseline (5\%)", "Baseline (10\%)", "Baseline (25\%)", "Baseline (50\%)", "Baseline (100\%)"], ha='left')
ax.get_yaxis().set_tick_params(pad=60)

legend = ax.get_legend()
legend.set_title("Simulator")
legend.get_texts()[0].set_text('Radice + OpenDC (v2.1)')
legend.get_texts()[1].set_text('CloudSim Plus (v7.1.1)')

fig.align_ylabels(ax)
fig.savefig(os.path.join(FIGURE_PATH, 'mem-comparison.pgf'))

In [ ]:
scales = {
    'baseline': 1,
    'baseline-50%': 0.5,
    'baseline-25%': 0.25,
    'baseline-10%': 0.1,
    'baseline-5%': 0.05
}

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=radice.figsize(2.2), sharex=True)

data = df[df['engine'] == 'opendc']
x = data['workload'].apply(lambda x: scales[x]).astype('float')
y = data['duration'].dt.seconds

sns.regplot(data=data, x=x, y=y, ax=ax[0], x_estimator=np.mean, order=2, x_ci='sd', scatter_kws={'zorder': 10})
sns.regplot(data=data, x=x[x > 0.1], y=y[x > 0.1], ax=ax[0], x_estimator=np.mean, x_ci='sd', scatter=False)

ax[0].set_xlabel("Workload Scale")
ax[0].set_ylabel("Runtime [\si{\second}]");

sns.regplot(data=data, x=x, y='mem_usage_mb', ax=ax[1], x_estimator=np.mean)

ax[1].set_xlabel("Workload Scale")
ax[1].xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax[1].yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
ax[1].set_ylabel("Memory Usage [\si{\mega\\byte}]");
ax[1].set_xlim([0, 1.04])
ax[1].set_ylim([0, 1000])

handles = [
    Line2D([0], [0], color='C0', label='Full'),
    Line2D([0], [0], color='C1', label='25%--100%'),
]

ax[0].legend(title='Linear Regression', handles=handles, loc='upper left', bbox_to_anchor=(0, 0.5, 0.53, 0.5), mode="expand")

fig.align_ylabels(ax)
fig.savefig(os.path.join(FIGURE_PATH, 'scaling-behavior.pgf'))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=radice.figsize(1.8), sharex=True)

data = df[df['engine'] == 'cloudsim-plus']
x = data['workload'].apply(lambda x: scales[x]).astype('float')

sns.regplot(data=data, x=x, y=data['duration'].dt.seconds, ax=ax[0], x_estimator=np.mean, order=2, x_ci='sd')

ax[0].set_xlabel("Workload Scale")
ax[0].set_ylabel("Runtime [\si{\second}]");

sns.regplot(data=data, x=x, y='mem_usage_mb', ax=ax[1], x_estimator=np.mean)

ax[1].set_xlabel("Workload Scale")
ax[1].xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax[1].set_ylabel("Memory Usage [\si{\mega\\byte}]");

fig.align_ylabels(ax)
fig.savefig(os.path.join(FIGURE_PATH, 'scaling-behavior-cloudsim-plus.pgf'))